In [2]:
from langchain_anthropic import ChatAnthropic

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.document_loaders import PyPDFLoader
#from typing import Listpy

In [66]:
with open("../api_key.txt") as f:
    api_key=f.read()


In [24]:
llm = ChatAnthropic(model="claude-3-sonnet-20240229",
                    temperature=0,
                    max_tokens=512,
                    timeout=None,
                    max_retries=2,
                    api_key=api_key)

In [38]:
file_path = "../Data/Personal Auto/Tesla Property & Casualty, Inc/TPC Rule Manual.pdf"
file_path_test = "../Data/Personal Auto/test/TX PPA APCIC Rules 101518RB.pdf"

loader = PyPDFLoader(file_path_test)
pages = loader.load_and_split()

this might not be a good idea due to pdf length

In [39]:
text = " ".join(list(map(lambda page: page.page_content, pages)))
len(text)

81014

TODO : we should split text and loop to respect model's context window

In [59]:
all_args = {
    "allow_inf_nan" : True,
    "default" : 0
    }

# default_factory -> callable when default is needed

class InsuranceExtraction(BaseModel):
    '''Different informations extracted from a compagny insurance's rule file.'''
    compagny_name: str = Field(description="The compagny that wrote this file")
    min_premium: str = Field(description="minimum premium in dollars, only one number", default=-1.0)
    min_premium_comments: str = Field(description="minimum premium commentaries or additionnal informations")
    policy_period: str = Field(description="policy period in month, only one number", default=-1.0)
    policy_period_comments: str = Field(description="policy period commentaries or additionnal informations")
    

In [60]:
structured_llm = llm.with_structured_output(InsuranceExtraction)

In [61]:
result = structured_llm.invoke(text)
result

InsuranceExtraction(compagny_name='Allied Property and Casualty Insurance Company', min_premium='-1.0', min_premium_comments='The minimum premium is not specified in the provided rule file.', policy_period='6 or 12', policy_period_comments='Premiums are developed based on a six-month policy period. For twelve month policies, the premium is modified per the rating sequence.')

test on document without the information -> error handling